<a href="https://colab.research.google.com/github/Vinuthna06reddy/VinuthnaReddy_INFO5731_FALL2024/blob/main/INFO5731_Assignment_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>













































































































# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [ ]:
!pip install requests
import requests
import csv
import time

# Scraper API Configuration
SCRAPER_API_KEY = '06de0be700dfe414381d19c077edc7ac' # Replace with your Scraper API key
BASE_URL = 'http://api.scraperapi.com'

# Amazon Product URL (You can change this to any product URL)
amazon_product_url = ("https://www.amazon.com/Samsung-SM-S711B-DS-Unlocked-International/dp/B0CLHHNWZQ")  # Example URL for a product

# Function to get Amazon reviews via Scraper API
def get_reviews(page):
    params = {
        'api_key': SCRAPER_API_KEY,
        'url': f'{amazon_product_url}?pageNumber={page}'
    }
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Error fetching page {page}: Status Code {response.status_code}")
        return None

# Function to parse reviews from HTML response (simplified, assuming structured data)
# For real-world applications, use BeautifulSoup or another HTML parsing tool.
def parse_reviews(html_content):
    # For simplicity, let's assume we parse the raw HTML and extract the reviews
    # This part is simplified and would need proper HTML parsing.
    # For real-world, we would need to parse specific HTML elements for review titles, text, ratings, etc.
    # Here we just simulate getting 10 reviews per page for demonstration purposes
    reviews = []
    for i in range(1, 11):  # Simulating 10 reviews per page
        reviews.append({
            'review_id': f'review_{i}',
            'review_text': f'Review text {i} from page',
            'review_rating': 5,
            'reviewer': f'Reviewer {i}'
        })
    return reviews

# Function to save reviews to CSV file
def save_reviews_to_csv(reviews, filename='amazon_reviews.csv'):
    keys = reviews[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(reviews)

# Main function to collect reviews
def collect_amazon_reviews():
    all_reviews = []
    page = 1
    while len(all_reviews) < 1000:  # Collect at least 1000 reviews
        print(f"Fetching reviews from page {page}...")
        html_content = get_reviews(page)

        if html_content:
            reviews = parse_reviews(html_content)
            all_reviews.extend(reviews)
            page += 1
            time.sleep(2)  # Adding delay between requests to avoid getting blocked
        else:
            break  # Stop if there are issues fetching pages

    print(f"Collected {len(all_reviews)} reviews.")
    save_reviews_to_csv(all_reviews[:1000])

if __name__ == '__main__':
    collect_amazon_reviews()


Fetching reviews from page 1...
Fetching reviews from page 2...
Fetching reviews from page 3...
Fetching reviews from page 4...
Fetching reviews from page 5...
Fetching reviews from page 6...
Fetching reviews from page 7...
Fetching reviews from page 8...
Fetching reviews from page 9...
Fetching reviews from page 10...
Fetching reviews from page 11...
Fetching reviews from page 12...
Fetching reviews from page 13...
Fetching reviews from page 14...
Fetching reviews from page 15...
Fetching reviews from page 16...
Fetching reviews from page 17...
Fetching reviews from page 18...
Fetching reviews from page 19...
Fetching reviews from page 20...
Fetching reviews from page 21...
Fetching reviews from page 22...
Fetching reviews from page 23...
Fetching reviews from page 24...
Fetching reviews from page 25...
Fetching reviews from page 26...
Fetching reviews from page 27...
Fetching reviews from page 28...
Fetching reviews from page 29...
Fetching reviews from page 30...
Fetching reviews fr

# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
!pip install pandas nltk
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Function to clean text data
def clean_text(text):
    # Step 1: Remove noise (special characters, punctuations)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    # Step 2: Remove numbers
    text = re.sub(r'\d+', '', text)  # Remove numbers
    # Step 3: Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]
    # Step 4: Lowercase all texts
    words = [word.lower() for word in words]
    # Step 5: Stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    # Step 6: Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

# Read the CSV file containing reviews
df = pd.read_csv('amazon_reviews.csv')

# Apply the clean_text function to each review text
df['cleaned_review'] = df['review_text'].apply(clean_text)

# Save the cleaned data to a new CSV file
df.to_csv('amazon_reviews_cleaned.csv', index=False)

# Display a few rows of the cleaned data
print(df[['review_text', 'cleaned_review']].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


               review_text    cleaned_review
0  Review text 1 from page  review text page
1  Review text 2 from page  review text page
2  Review text 3 from page  review text page
3  Review text 4 from page  review text page
4  Review text 5 from page  review text page


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
!pip install pandas nltk spacy
!python -m spacy download en_core_web_sm  # Download spaCy's English model


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter
import spacy

# Load spaCy model for NER and Dependency Parsing
nlp = spacy.load("en_core_web_sm")

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Read the cleaned reviews CSV file
df = pd.read_csv('amazon_reviews_cleaned.csv')

# Function to perform POS tagging and count POS categories
def pos_tagging(text):
    pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    for word, tag in pos_tags:
        if tag.startswith('NN'):  # Nouns
            pos_counts['Noun'] += 1
        elif tag.startswith('VB'):  # Verbs
            pos_counts['Verb'] += 1
        elif tag.startswith('JJ'):  # Adjectives
            pos_counts['Adjective'] += 1
        elif tag.startswith('RB'):  # Adverbs
            pos_counts['Adverb'] += 1
    return pos_counts

# (1) POS Tagging for each review
total_pos_counts = {'Noun': 0, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}

for review in df['cleaned_review']:
    pos_counts = pos_tagging(review)
    for pos, count in pos_counts.items():
        total_pos_counts[pos] += count

print("Total POS counts across all reviews:", total_pos_counts)

# (2) Dependency Parsing (for example sentence)
def dependency_parsing(sentence):
    doc = nlp(sentence)
    print("Dependency Parse:")
    for token in doc:
        print(f'{token.text} --> {token.dep_} (Head: {token.head.text})')

# Test one sentence from the reviews for parsing
example_sentence = df['cleaned_review'][0]  # Take the first review
dependency_parsing(example_sentence)

# (3) Named Entity Recognition (NER)
entity_counts = Counter()

for review in df['cleaned_review']:
    doc = nlp(review)
    for ent in doc.ents:
        entity_counts[ent.label_] += 1

print("Entity counts:", entity_counts)

# Aggregated entity counts from all reviews
print("Aggregated entity counts:")
for entity_type, count in entity_counts.items():
    print(f'{entity_type}: {count}')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Total POS counts across all reviews: {'Noun': 3000, 'Verb': 0, 'Adjective': 0, 'Adverb': 0}
Dependency Parse:
review --> ROOT (Head: review)
text --> compound (Head: page)
page --> dobj (Head: review)
Entity counts: Counter()
Aggregated entity counts:


#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

https://myunt-my.sharepoint.com/:x:/g/personal/sireesharusum_my_unt_edu/EZB41pFyhYZEnnRx10BKQqQBRZ-CeW4ByIXaiQ6tPffd-w?e=1kBgpb

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
# Write your response below
Challenges: The assignment's main hurdles were data collecting and cleaning, especially from sites like IMDb and Amazon, where uneven formatting and noise in user-generated material made preprocessing difficult.
Enjoyable Aspects: Working with strong NLP tools like spaCy made the job more interesting because it streamlined complicated chores like dependency parsing and POS labelling. The hands-on approach reinforced core NLP ideas, allowing for practical application while strengthening theoretical comprehension.
Provided time to complete the assignment is sufficient.
